
# Tinkoff sirius ML 2023 RecSys intro task (Модели)

## 🛑 Attention! 🛑

Дорогие проверяющие, я отправил работу до 1-го сентября, после узнал, что дедлайн перенесли, пожалуйста, вернитесь сюда после нового дедлайна (8-го сентября), это того стоит.

### Бибилиотеки, данные, метрики, класс для наследования

In [1]:
!pip install implicit >> /dev/null
!pip install catboost >> /dev/null

In [2]:
from abc import ABC, abstractmethod
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import implicit
from scipy.sparse import csr_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import ParameterGrid

In [3]:
data_folder = '/content/drive/MyDrive/kion_data_featured/'

users_df = pd.read_csv(data_folder + 'users.csv')
items_df = pd.read_csv(data_folder + 'items.csv')
train_part = pd.read_csv(data_folder + 'train.csv', parse_dates=["last_watch_dt"])
test_part = pd.read_csv(data_folder + 'test_data.csv')
test_part = test_part.groupby("user_id").agg({"ground_truth": list}).reset_index()

In [4]:
 # https://github.com/deethereal/tinkoff-sirius-ml-2023-recsys-intro-task/blob/main/intro_task.ipynb
 # ACHTUNG! DO NOT TOUCH

def ndcg_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    at = len(predicted)
    relevance = np.array([1 if x in predicted else 0 for x in gt_items])
    # DCG uses the relevance of the recommended items
    rank_dcg = dcg(relevance)
    if rank_dcg == 0.0:
        return 0.0

    # IDCG has all relevances to 1 (or the values provided), up to the number of items in the test set that can fit in the list length
    ideal_dcg = dcg(np.sort(relevance)[::-1][:at])

    if ideal_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg

    return ndcg_


def dcg(scores: np.ndarray) -> float:
    return np.sum(
        np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)), dtype=np.float64
    )


def recall_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    n_gt = len(gt_items)
    intersection = len(set(gt_items).intersection(set(predicted)))
    return intersection / n_gt


def evaluate_recommender(df: pd.DataFrame, model_preds_col: str, gt_col: str = "ground_truth") -> Dict[str, float]:
    metric_values = []

    for _, row in df.iterrows():
        metric_values.append(
            (ndcg_metric(row[gt_col], row[model_preds_col]), recall_metric(row[gt_col], row[model_preds_col]))
        )

    return {"ndcg": np.mean([x[0] for x in metric_values]), "recall": np.mean([x[1] for x in metric_values])}

In [5]:
# https://github.com/deethereal/tinkoff-sirius-ml-2023-recsys-intro-task/blob/main/intro_task.ipynb
class BaseRecommender(ABC):
    def __init__(self):
        self.trained = False

    @abstractmethod
    def fit(self, df: pd.DataFrame) -> None:
        # реализация может быть любой, никаких ограничений

        # не забудьте про
        self.trained = True

    @abstractmethod
    def predict(self, df: pd.DataFrame, topn: int = 10) -> List[np.ndarray]:
        # реализация может быть любой, НО
        # должен возвращать список массивов из item_id, которые есть в `item_df`, чтобы корректно работал подсчет метрик
        pass

## Cтатистически подходы

### Топ популярных (Baseline)

In [6]:
# https://github.com/deethereal/tinkoff-sirius-ml-2023-recsys-intro-task/blob/main/intro_task.ipynb
class TopPopular(BaseRecommender):
    def __init__(self):
        super().__init__()

    def fit(self, df: pd.DataFrame, item_id_col: str = "item_id") -> None:
        # считаем популярность айтемов
        self.recommendations = df[item_id_col].value_counts().index.values
        self.trained = True

    def predict(self, df: pd.DataFrame, topn: int = 10) -> np.ndarray:
        assert self.trained
        # возвращаем для всех одно и то же
        return [self.recommendations[:topn]] * len(df)


toppop = TopPopular()
toppop.fit(train_part)
test_part["toppopular_recs"] = toppop.predict(test_part)
evaluate_recommender(df=test_part, model_preds_col="toppopular_recs")

{'ndcg': 0.17037237918248196, 'recall': 0.0763696799665908}

### Топ популярных для разных категорий

Во время EDA мы выделили новые признаки для фильмов, а именно: какого пола человека чаще проссматривает фильм, какого возраста и дохода. Эти признаки будем использовать для рекомендаций. Для пользователей, про которых мы ничего не знаем (они есть в train_part, но их нет в users_df) - будем рекомендовать популярные.

In [7]:
class CategoryTopPopularRecommender(BaseRecommender):
    def __init__(self, category: str):
        super().__init__()
        self.cat_popularity = None
        self.trained = False
        self.top_popular_items = None
        self.category = category

    def fit(self, train_part: pd.DataFrame, users_df: pd.DataFrame, items_df: pd.DataFrame,
            item_id_col = 'item_id', views_col = 'views', user_id_col='user_id'):
        # Вычисляем общий список популярных фильмов по все значениям категории
        self.top_popular_items = items_df[[item_id_col, views_col]]\
                                  .sort_values(views_col, ascending=False)[item_id_col].to_list()

        # Группируем 'items_df' по категории и сортируем фильмы по популярности (колонка 'views')
        self.cat_popularity = dict(train_part.merge(users_df, how='left', on=user_id_col)[[self.category, item_id_col]]\
                                   .groupby(self.category)[item_id_col]\
                                   .apply(lambda x: x.value_counts().index.tolist()))


        self.trained = True

    def predict(self, test_part: pd.DataFrame, users_df: pd.DataFrame, user_id_col = 'user_id', topn=10):
        assert self.trained

        # Объединяем 'test_part' с 'users_df' по 'user_id' и оставляем только 'user_id' и категорию
        user_data = users_df[[user_id_col, self.category]]
        test_data = test_part.merge(user_data, on='user_id', how='left')

        # Создаем пустой столбец 'recommendations' для хранения рекомендаций
        test_data['recommendations'] = None

        # Для каждой строки в test_data вычисляем рекомендации
        for index, row in test_data.iterrows():
            cat = row[self.category]
            if cat in self.cat_popularity:
                recommendations = self.cat_popularity[cat][:topn]
            # Если мы ничего не знаем про пользователя, то рекомендуем популярное
            else:
                recommendations = self.top_popular_items[:topn]

            test_data.at[index, 'recommendations'] = recommendations

        # Возвращаем столбец 'recommendations' с рекомендациями
        return test_data['recommendations']

for cat in ['sex', 'income', 'age']:
    recommender = CategoryTopPopularRecommender(cat)
    recommender.fit(train_part, users_df, items_df)
    test_part[cat + "_recs"] = recommender.predict(test_part, users_df)
    print(f'Разибение по {cat}: ', evaluate_recommender(df=test_part, model_preds_col=cat + "_recs"))

Разибение по sex:  {'ndcg': 0.1712007695565208, 'recall': 0.07668173604995636}
Разибение по income:  {'ndcg': 0.17148589785741739, 'recall': 0.07695524790918498}
Разибение по age:  {'ndcg': 0.17224145285241338, 'recall': 0.07724899822276024}


Видим, что разбиение по категориям работает немного лучше, чем бейзлайн.

## Классические методы (Implicit)



Создадим класс для работы с implicit, для того чтбы не повторять код. От него будем создавать все implicit модели.

In [8]:
# https://github.com/deethereal/tinkoff-sirius-ml-2023-recsys-intro-task/blob/main/intro_task.ipynb

# Изначально я написал свою реализацию класса, она получилось сложной и запутанной,
# поэтому решил позаимствовать реализвацию из условия задания

class ImplicitBaseRecommender(BaseRecommender):
    def __init__(self, model=None, **kwargs) -> None:
        super().__init__()
        self.params = kwargs
        self.model = model(**kwargs)
        self.trained = False

    def fit(
        self, df: pd.DataFrame, item_col: str = "item_id", user_col: str = "user_id", value_col: str = None
    ) -> None:
        self.user_encoder = LabelEncoder()
        self.item_encoder = LabelEncoder()
        user_ids = self.user_encoder.fit_transform(df[user_col])
        item_ids = self.item_encoder.fit_transform(df[item_col])
        if value_col is None:
            counts = np.ones(len(df))
        else:
            counts = df[value_col].values

        matrix_shape = len(self.user_encoder.classes_), len(self.item_encoder.classes_)

        self.sparse = csr_matrix((counts, (user_ids, item_ids)), shape=matrix_shape)
        self.model.fit(self.sparse)

        self.trained = True

    def predict(self, df: pd.DataFrame, topn: int = 10) -> List[np.ndarray]:
        assert self.trained

        all_recs = []

        users = self.user_encoder.transform(df["user_id"])
        for user in tqdm(users, desc="predicting", leave=False):
            encoded_rec_items = self.model.recommend(user, user_items=self.sparse[user], N=topn)[0]
            all_recs.append(self.item_encoder.inverse_transform(encoded_rec_items))

        return all_recs

    # два метода, чтобы правильно работал подбор гиппер параматеров через кроссвалидацию из sklearn
    # https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html
    #
    # В итоге было принято решение отказаться от подбора гиперпараметров c помощью k-fold cv подробнее ниже
    def set_params(self, **params):
        self.model = self.model(**params)
        return self

    def get_params(self, deep=True):
        return {**self.params}

### kNN

In [9]:
knn_rec = ImplicitBaseRecommender(model=implicit.nearest_neighbours.CosineRecommender, K=15_000)
knn_rec.fit(train_part)
test_part["knn_recs"] = knn_rec.predict(test_part)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.011420011520385742 seconds
  warnings.warn(


  0%|          | 0/12067 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

In [10]:
evaluate_recommender(test_part, model_preds_col="knn_recs")

{'ndcg': 0.12234769080835413, 'recall': 0.054391934978456756}

Видим, что качество просело даже по сравнению с бейзлайном.

### TFIDF

In [11]:
tfidf_rec = ImplicitBaseRecommender(model=implicit.nearest_neighbours.TFIDFRecommender, K=15_000)
tfidf_rec.fit(train_part)
test_part["tfidf_recs"] = tfidf_rec.predict(test_part)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.011855840682983398 seconds
  warnings.warn(


  0%|          | 0/12067 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

In [12]:
evaluate_recommender(test_part, model_preds_col="tfidf_recs")

{'ndcg': 0.12740156038704564, 'recall': 0.05658684677730701}

Аналогичная картина.

### Item to item

In [13]:
itemitem_rec = ImplicitBaseRecommender(model=implicit.nearest_neighbours.ItemItemRecommender, K=10_000)
itemitem_rec.fit(train_part)
test_part["itemitem_recs"] = itemitem_rec.predict(test_part)

  0%|          | 0/12067 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

In [14]:
evaluate_recommender(test_part, model_preds_col="itemitem_recs")

{'ndcg': 0.191239213767931, 'recall': 0.08579517402016573}

Item to Item сработал уже лучше, чем разделение на категории. Но и этот результат попробуем улучшить.

### ALS и подбор параметров через валидационную выборку

В ALS (и в моделях рассмотренных дальше) уже много параметров, и простое увеличение этих параметров не приведет к увеличению качества (как было с количеством соседей). Поэтому нам нужно использовать кросс-валидацию для подбора этих параметров.

Далее я столкнулся с проблемой: если использовать k-fold кросс-валидацию, то слишком много данных будет в тестовой часте, как следствие, модель не сможет хорошо обучиться. Эту проблему можно решить, если использовать большое количество фолдов, либо вообще leave-one-out, но у меня нет таких мощностей. *(По моим подсчетам на 50 фолдов и 25 наборов параметров нужно примерно 8 часов с использованием gpu)*

Будем использовать просто Hold-out валидацию на тестовой выборке.

---
https://datascience.stackexchange.com/questions/6814/how-to-split-train-test-in-recommender-systems

https://academy.yandex.ru/handbook/ml/article/kross-validaciya

https://github.com/anamarina/RecSys_course



Будем использовать gpu google colab`а, это сэкономит кучу времени. Проверим cuda.

In [15]:
!nvidia-smi

Tue Sep  5 19:09:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Реализуем функцию для валидации параметров, использующую Hold Out валидацию.

In [16]:
# https://tqdm.github.io/
# https://github.com/scikit-learn/scikit-learn/blob/7f9bad99d/sklearn/model_selection/_search.py#L50

# Про реализованную функцию len() узнал из исходного кода, в документации про неё ничего нет
# функцию len() вызывает tqdm
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html#sklearn.model_selection.ParameterGrid

def ImplicitHoldOutCV(model,
              train_part: pd.DataFrame, test_part: pd.DataFrame,
              params: dict[str, list]
) -> tuple[:, dict[str, int], dict[str, int]]:
    param_grid = ParameterGrid(params)

    best_model = None
    best_params: dict = None
    best_score: dict[str, int] = {'ndcg': 0, 'recall': 0}

    for params in tqdm(param_grid):
        rec_model = ImplicitBaseRecommender(model=model, **params)
        rec_model.fit(train_part)
        test_part["recs"] = rec_model.predict(test_part)
        score = evaluate_recommender(test_part, model_preds_col="recs")
        if score['ndcg'] > best_score['ndcg'] \
                and score['recall'] > best_score['recall']:
            best_score = score
            best_model = rec_model
            best_params = params

    return best_model, best_params, best_score

In [17]:
# Параметры, которые использовались изначально
# params = {
#         'factors': [1, 2, 3],
#         'regularization': [0.01, 0.02],
#         'alpha': [0.1, 0.075, 0.125],
#         'iterations': [1, 2, 5, 10, 15],
#         'random_state': [42]
# }


# Лучшие параметры
params = {
    'alpha': [0.001],
    'factors': [1],
    'iterations': [5],
    'random_state': [42],
    'regularization': [0.01]
}

# model = implicit.gpu.als.AlternatingLeastSquares
model = implicit.cpu.als.AlternatingLeastSquares

best_als_model, best_als_params, best_als_score = ImplicitHoldOutCV(model,
                                                                    train_part,
                                                                    test_part,
                                                                    params)

test_part["als_recs"] = best_als_model.predict(test_part)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

In [18]:
best_als_score

{'ndcg': 0.18147419856444427, 'recall': 0.0822750659878154}

Item to item мы не превзошли, но лучше чем бейзлайн и топ популярных по категориям.

### BM25

BM25 увидел в документации [implicit](https://benfred.github.io/implicit/api/models/cpu/knn.html?highlight=bm25#implicit.nearest_neighbours.BM25Recommender), подробнее прочитал [здесь](https://medium.com/@evertongomede/understanding-the-bm25-ranking-algorithm-19f6d45c6ce).

In [19]:
# Параметры, которые использовались изначально
# params = {
#         'K': [15_000],
#         'K1': [0.0, 0.1, 0.4, 0.6, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.5],
#         'B': [0, 0.25, 0.4, 0.7, 0.75, 0.85, 0.1, 0.11, 0.125]
# }


# Лучшие параметры
params = {
        'K': [15_000],
        'K1': [0],
        'B': [0]
}

model = implicit.nearest_neighbours.BM25Recommender

best_bm25_model, best_bm25_params, best_bm25_score = ImplicitHoldOutCV(model,
                                                               train_part, test_part,
                                                               params)

test_part["bm25_recs"] = best_bm25_model.predict(test_part)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.015950918197631836 seconds
  warnings.warn(


  0%|          | 0/12067 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

In [20]:
best_bm25_score

{'ndcg': 0.1923119207092606, 'recall': 0.08640752850098624}

### BPR


In [22]:
# Параметры, которые использовались изначально
params = {
        'factors': [100, 300, 400, 600],
        'regularization': [0.01, 0.02, 0.05, 0.075],
        'learning_rate': [0.001, 0.01, 0.075],
        'iterations': [10, 25, 30, 40],
        'random_state': [42]
}


# Лучшие параметры
params = {
         'factors': [400],
         'iterations': [10],
         'learning_rate': [0.001],
         'random_state': [42],
         'regularization': [0.05]
}

model = implicit.gpu.bpr.BayesianPersonalizedRanking
# model = implicit.cpu.bpr.BayesianPersonalizedRanking

best_bpr_model, best_bpr_params, best_bpr_score = ImplicitHoldOutCV(model,
                                                                    train_part, test_part,
                                                                    params)

test_part["bpr_recs"] = best_bpr_model.predict(test_part)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

predicting:   0%|          | 0/18693 [00:00<?, ?it/s]

In [23]:
best_bpr_score

{'ndcg': 0.07815303702218993, 'recall': 0.031093927503052367}

### LMF

In [ ]:
# Параметры, которые использовались изначально
# params = {
#         'factors': [30, 40, 50, 70, 80],
#         'regularization': [0.1, 0.2, 0.5, 0.75, 1],
#         'learning_rate': [0.001, 0.01, 0.075, 0.125],
#         'iterations': [40, 50, 80, 100],
#         'neg_prop' : [5, 10, 30, 50],
#         'random_state': [42]
# }


# Лучшие параметры
params = {'factors': [40],
         'iterations': [300],
         'learning_rate': [0.3],
         'neg_prop': [5],
         'random_state': [42],
         'regularization': [1]
}

model = implicit.cpu.lmf.LogisticMatrixFactorization

best_lmf_model, best_lmf_params, best_lmf_score = ImplicitHoldOutCV(model,
                                                                    train_part, test_part,
                                                                    params)

test_part["lmf_recs"] = best_lmf_model.predict(test_part)

In [ ]:
best_lmf_score

## Boosting is all you need

Такое большое количество моделей здесь и признаков в EDA части были созданы как раз для использования бустинга.

Будем использовать предсказание лучшей классической модели в качестве признака для бустинга. Получим двухэтапную модель, в которой на первом месте будет item to item из implicit, на втором catboost.